**Exoplanet Transit Fitting with Kepler/TESS Data**

In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Exoplanet Transit Analysis: BLS + BATMAN Fit\n",
    "\n",
    "This notebook:\n",
    "1. Downloads Kepler/TESS light curve data using **Lightkurve**.\n",
    "2. Runs a **Box Least Squares (BLS)** search to detect transit period.\n",
    "3. Uses **BATMAN** + **emcee** to fit the transit and estimate `Rp/Rs`.\n",
    "\n",
    "👉 Change the `target_name` and `mission` in the first cell to analyze a different star."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "!pip install lightkurve astropy batman-package emcee corner tqdm"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part A — BLS period search"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import lightkurve as lk\n",
    "from astropy.timeseries import BoxLeastSquares\n",
    "\n",
    "# ===== USER PARAMETERS =====\n",
    "target_name = \"Kepler-10\"   # change to star name or TIC ID\n",
    "mission = \"Kepler\"          # \"Kepler\" or \"TESS\"\n",
    "search_index = 0            # if multiple results, choose index\n",
    "min_period, max_period = 0.2, 30.0  # days\n",
    "freq_factor = 5.0\n",
    "q_min, q_max = 0.001, 0.1\n",
    "flatten_window = 401\n",
    "# ===========================\n",
    "\n",
    "# 1. Search + download\n",
    "search = lk.search_lightcurve(target_name, mission=mission)\n",
    "print(search[:5])\n",
    "lc = search[search_index].download()\n",
    "if lc is None:\n",
    "    raise ValueError(\"No LC found. Try different search index/target.\")\n",
    "\n",
    "# 2. Clean + flatten\n",
    "lc = lc.remove_nans().remove_outliers(sigma=5)\n",
    "lc_flat = lc.flatten(window_length=flatten_window).normalize()\n",
    "\n",
    "time, flux = lc_flat.time.value, lc_flat.flux.value\n",
    "\n",
    "# 3. Run BLS\n",
    "bls = BoxLeastSquares(time, flux)\n",
    "periods = np.exp(np.linspace(np.log(min_period), np.log(max_period), int((max_period/min_period)*freq_factor)))\n",
    "bls_power = bls.power(periods, q_min, q_max)\n",
    "\n",
    "best_idx = np.argmax(bls_power.power)\n",
    "best_period = bls_power.period[best_idx]\n",
    "best_t0 = bls_power.transit_time[best_idx]\n",
    "best_dur = bls_power.duration[best_idx]\n",
    "print(f\"Best period = {best_period:.6f} d, duration = {best_dur:.4f} d, t0 = {best_t0:.4f}\")\n",
    "\n",
    "# 4. Plot BLS power\n",
    "plt.figure(figsize=(10,3))\n",
    "plt.plot(bls_power.period, bls_power.power, 'k-')\n",
    "plt.axvline(best_period, color='C1', label=f\"P={best_period:.5f} d\")\n",
    "plt.xscale('log')\n",
    "plt.xlabel('Period (days)'); plt.ylabel('BLS power'); plt.legend();\n",
    "plt.title(f'BLS Periodogram: {target_name}')\n",
    "plt.show()\n",
    "\n",
    "# 5. Folded LC\n",
    "fold = lc_flat.fold(period=best_period, t0=best_t0)\n",
    "plt.figure(figsize=(10,4))\n",
    "fold.scatter(alpha=0.3, s=5)\n",
    "fold.bin(bins=200).scatter(color='red', s=20)\n",
    "plt.xlabel('Phase (days)'); plt.ylabel('Normalized flux')\n",
    "plt.title(f'Folded LC at P={best_period:.5f} d')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Part B — Transit fit with BATMAN + emcee"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "import batman, emcee, corner\n",
    "from tqdm.auto import tqdm\n",
    "\n",
    "# ===== Priors & settings =====\n",
    "period = best_period\n",
    "t0_epoch = best_t0\n",
    "a_prior_center, a_prior_sigma = 15.0, 5.0\n",
    "inc_prior_center, inc_prior_sigma = 87.0, 3.0\n",
    "ld_u, ld_model = [0.1, 0.3], \"quadratic\"\n",
    "fit_window = 0.12\n",
    "nwalkers, nsteps, burn = 40, 1500, 400\n",
    "# =============================\n",
    "\n",
    "# Select transit window\n",
    "fold = lc_flat.fold(period=period, t0=t0_epoch)\n",
    "t_all = fold.time.value * period\n",
    "f_all = fold.flux.value\n",
    "mask = (t_all > -fit_window*period) & (t_all < fit_window*period)\n",
    "times, fluxes = t_all[mask], f_all[mask]\n",
    "flux_err = np.full_like(fluxes, np.nanstd(fluxes))\n",
    "\n",
    "# Batman helpers\n",
    "def make_params(per, t0, rp, a, inc):\n",
    "    p = batman.TransitParams()\n",
    "    p.t0, p.per, p.rp, p.a, p.inc = t0, per, rp, a, inc\n",
    "    p.ecc, p.w = 0.0, 90.0\n",
    "    p.limb_dark, p.u = ld_model, ld_u\n",
    "    return p\n",
    "def batman_flux(times, per, t0, rp, a, inc):\n",
    "    p = make_params(per, t0, rp, a, inc)\n",
    "    return batman.TransitModel(p, times).light_curve(p)\n",
    "\n",
    "# log-probability\n",
    "def log_prior(theta):\n",
    "    rp, t0_off, a, inc, baseline = theta\n",
    "    if not (0.0001 < rp < 0.5 and -0.2*period < t0_off < 0.2*period and 1 < a < 500 and 70 < inc < 90 and 0.8 < baseline < 1.2):\n",
    "        return -np.inf\n",
    "    return -0.5 * ((a - a_prior_center)**2/a_prior_sigma**2 + (inc - inc_prior_center)**2/inc_prior_sigma**2)\n",
    "def log_likelihood(theta):\n",
    "    rp, t0_off, a, inc, baseline = theta\n",
    "    model = baseline * batman_flux(times, period, t0_epoch + t0_off, rp, a, inc)\n",
    "    return -0.5 * np.sum(((fluxes - model)/flux_err)**2 + np.log(2*np.pi*flux_err**2))\n",
    "def log_prob(theta):\n",
    "    lp = log_prior(theta)\n",
    "    return lp + log_likelihood(theta) if np.isfinite(lp) else -np.inf\n",
    "\n",
    "# Run emcee\n",
    "ndim = 5\n",
    "p0 = [0.1, 0.0, a_prior_center, inc_prior_center, 1.0]\n",
    "pos = p0 + 1e-3 * np.random.randn(nwalkers, ndim)\n",
    "sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob)\n",
    "sampler.run_mcmc(pos, nsteps, progress=True)\n",
    "\n",
    "samples = sampler.get_chain(discard=burn, thin=10, flat=True)\n",
    "labels = [\"rp\", \"t0_off\", \"a\", \"inc\", \"baseline\"]\n",
    "\n",
    "# Print results\n",
    "for i, lab in enumerate(labels):\n",
    "    p16, p50, p84 = np.percentile(samples[:, i], [16,50,84])\n",
    "    print(f\"{lab} = {p50:.4f} (+{p84-p50:.4f}/-{p50-p16:.4f})\")\n",
    "\n",
    "# Corner plot\n",
    "corner.corner(samples, labels=labels, show_titles=True);\n",
    "\n",
    "# Best-fit model vs data\n",
    "best = np.median(samples, axis=0)\n",
    "rp_b, t0off_b, a_b, inc_b, baseline_b = best\n",
    "model = baseline_b * batman_flux(times, period, t0_epoch + t0off_b, rp_b, a_b, inc_b)\n",
    "\n",
    "plt.figure(figsize=(10,4))\n",
    "plt.scatter(times, fluxes, s=10, alpha=0.5)\n",
    "plt.plot(times, model, 'r-', lw=2, label='Best-fit')\n",
    "plt.xlabel('Time (days)'); plt.ylabel('Normalized flux')\n",
    "plt.title(f\"Transit Fit: Rp/Rs ≈ {rp_b:.3f}\")\n",
    "plt.legend(); plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Exoplanet Transit Analysis: BLS + BATMAN Fit\n',
    '\n',
    'This notebook:\n',
    '1. Downloads Kepler/TESS light curve data using **Lightkurve**.\n',
    '2. Runs a **Box Least Squares (BLS)** search to detect transit period.\n',
    '3. Uses **BATMAN** + **emcee** to fit the transit and estimate `Rp/Rs`.\n',
    '\n',
    '👉 Change the `target_name` and `mission` in the first cell to analyze a different star.']},
  {'cell_type': 'code',
   'metadata': {},
   'source': ['!pip install lightkurve astropy batman-package emcee corner tqdm']},
  {'cell_type': 'markdown',
   'metadata': {},
   'source': ['## Part A — BLS period search']},
  {'cell_type': 'code',
   'metadata': {},
   'source': ['import numpy as np\n',
    'import matplotlib.pyplot as plt\n',
    'import lightkurve as lk\n',
    'from astropy.timeseries import BoxLeastSquares\n',
    '\n',
    '# ===== USER PARAMETERS =====\n',
    'target_name = "